# Test task

### All necessary imports for the tasks

In [1]:
import psycopg2
import pandas as pd
from datetime import datetime, timedelta

/var/folders/tj/z_xsyk816vz7_vpx5pyp1db40000gn/T/ipykernel_41790/1460747103.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## `Task 3`

In [2]:
connection = psycopg2.connect(
    dbname="statistic",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

cursor = connection.cursor()

In [3]:
cursor.execute("""
    SELECT issue_key, author_key, duration_in_seconds
    FROM task_status_duration_seconds
    WHERE status = 'In Progress'
""")

In [4]:
def seconds_to_working_hours(seconds):
    workday_start = 10
    workday_end = 20
    seconds_in_hour = 3600
    hours_total = seconds / seconds_in_hour

    full_days = hours_total // (workday_end - workday_start)
    remaining_hours = hours_total % (workday_end - workday_start)

    start_date = datetime(2024, 6, 10, workday_start)
    current = start_date
    hours_count = 0

    while hours_count < remaining_hours:
        if current.weekday() < 5:
            work_end = datetime(
                current.year, current.month, current.day,
                workday_end
            )
            hours_count += (work_end - current).total_seconds() / seconds_in_hour

        current += timedelta(hours=24)

    return round(full_days * 8 + remaining_hours, 2)



In [5]:
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=['issue_key', 'author_key', 'duration_seconds'])
df['working_hours'] = df['duration_seconds'].apply(seconds_to_working_hours)
df

,issue_key,author_key,duration_seconds,working_hours
0,PYTHON-1003,jesse,2371.000000,0.66
1,PYTHON-1003,jesse,1487725.000000,331.26
2,PYTHON-1006,behackett,3575048.000000,795.07
3,PYTHON-1009,behackett,1632189.000000,363.39
4,PYTHON-1010,anna.herlihy@10gen.com,248327.000000,56.98
...,...,...,...,...
2614,PYTHON-996,behackett,11856977.000000,2635.60
2615,PYTHON-997,behackett,3401795.000000,756.94
2616,PYTHON-997,behackett,954466.000000,213.13
2617,PYTHON-999,luke.lovett,73546.000000,16.43


## `Task 4`

In [6]:
df['adjusted_working_hours'] = df['working_hours']
author_number_of_times = df['author_key'].value_counts()

for index, row in df.iterrows():
    author_key = row['author_key']
    n = author_number_of_times[author_key]
    df.at[index, 'adjusted_working_hours'] = row['working_hours'] / n
df

,issue_key,author_key,duration_seconds,working_hours,adjusted_working_hours
0,PYTHON-1003,jesse,2371.000000,0.66,0.002101910828025477707006369427
1,PYTHON-1003,jesse,1487725.000000,331.26,1.054968152866242038216560510
2,PYTHON-1006,behackett,3575048.000000,795.07,0.5893773165307635285396590067
3,PYTHON-1009,behackett,1632189.000000,363.39,0.2693773165307635285396590067
4,PYTHON-1010,anna.herlihy@10gen.com,248327.000000,56.98,0.5586274509803921568627450980
...,...,...,...,...,...
2614,PYTHON-996,behackett,11856977.000000,2635.60,1.953743513713862120088954781
2615,PYTHON-997,behackett,3401795.000000,756.94,0.5611119347664936990363232024
2616,PYTHON-997,behackett,954466.000000,213.13,0.1579911045218680504077094144
2617,PYTHON-999,luke.lovett,73546.000000,16.43,0.1039873417721518987341772152


## `Task 5`

In [7]:
cursor.execute("""
    SELECT issue_key, author_key, duration_in_seconds
    FROM task_status_duration_seconds
""")
rows = cursor.fetchall()
new_df = pd.DataFrame(rows, columns=['issue_key', 'author_key', 'duration_seconds'])
new_df['working_hours'] = new_df['duration_seconds'].apply(seconds_to_working_hours)
new_df['adjusted_working_hours'] = new_df['working_hours']

author_counts = new_df['author_key'].value_counts()
new_df.dropna(subset=['author_key'], inplace=True)

for index, row in new_df.iterrows():
    author_key = row['author_key']
    n = author_counts[author_key]
    new_df.at[index, 'adjusted_working_hours'] = row['working_hours'] / n
new_df

,issue_key,author_key,duration_seconds,working_hours,adjusted_working_hours
0,PYTHON-1,mike,470821908.238331,104627.86,675.0184516129032258064516129
1,PYTHON-10,mike,474009064.238331,105337.18,679.5947096774193548387096774
2,PYTHON-100,mike,435219190.238331,96716.22,623.9756129032258064516129032
3,PYTHON-1000,behackett,274341612.238331,60966.00,11.81282697151714783956597559
4,PYTHON-1001,behackett,274339728.238331,60965.48,11.81272621584964154233675644
...,...,...,...,...,...
13628,PYTHON-999,luke.lovett,73546.000000,16.43,0.04191326530612244897959183673
13629,PYTHON-999,luke.lovett,2344306.000000,521.20,1.329591836734693877551020408
13630,PYTHON-999,luke.lovett,2337976.000000,521.44,1.330204081632653061224489796
13631,PYTHON-999,luke.lovett,274246974.238331,60945.72,155.4737755102040816326530612


In [8]:
def calculate_cost(row):
    adjusted_hours = float(row['adjusted_working_hours'])
    if row['author_key'].startswith('JIRAUSER'):
        return adjusted_hours * 1.00
    else:
        return adjusted_hours * 2.00

new_df['cost'] = new_df.apply(calculate_cost, axis=1)
new_df

,issue_key,author_key,duration_seconds,working_hours,adjusted_working_hours,cost
0,PYTHON-1,mike,470821908.238331,104627.86,675.0184516129032258064516129,1350.036903
1,PYTHON-10,mike,474009064.238331,105337.18,679.5947096774193548387096774,1359.189419
2,PYTHON-100,mike,435219190.238331,96716.22,623.9756129032258064516129032,1247.951226
3,PYTHON-1000,behackett,274341612.238331,60966.00,11.81282697151714783956597559,23.625654
4,PYTHON-1001,behackett,274339728.238331,60965.48,11.81272621584964154233675644,23.625452
...,...,...,...,...,...,...
13628,PYTHON-999,luke.lovett,73546.000000,16.43,0.04191326530612244897959183673,0.083827
13629,PYTHON-999,luke.lovett,2344306.000000,521.20,1.329591836734693877551020408,2.659184
13630,PYTHON-999,luke.lovett,2337976.000000,521.44,1.330204081632653061224489796,2.660408
13631,PYTHON-999,luke.lovett,274246974.238331,60945.72,155.4737755102040816326530612,310.947551


### Total cost within the task type

In [9]:
result = new_df.groupby('issue_key')['cost'].sum().reset_index()
result.columns = ['issue_key', 'total_cost']

result

,issue_key,total_cost
0,PYTHON-1,1350.036903
1,PYTHON-10,1359.189419
2,PYTHON-100,1247.951226
3,PYTHON-1000,23.625654
4,PYTHON-1001,23.625452
...,...,...
2855,PYTHON-992,338.025180
2856,PYTHON-995,23.776020
2857,PYTHON-996,47.420217
2858,PYTHON-997,47.669917


### Thats all, thanks for viewing!